# Weather Underground json to csv Data Parser

![WU Logo](wu_logo.jpg)

## 1. Introduction

This notebook provides a parser to .json files obtained from Weather Underground (WU) API. If you do not have the .json files, use the `Weather Underground API json Data Acquisition Notebook` in this same project, and then return to this Notebook.

This parser will take as input the folder containing all json files, and output a single .csv file containing all the readings. 

The raw data contains the timestamps in UTC and a guessed local time which the criteria is not completely clear. As such, we will use the timestamp in UTC, but covert it in this notebook to HST to ensure consistency. 

As discussed in Section 2 - Data Model below, each set of readings share a common timestamp. As such, the output format of this notebook is a wide format table, i.e., each column is a type of reading (e.g. temperature, releative humidity, etc), and every row is identified by a timestamp shared by these different readings. 

**Terms of Use of WU Data (not of this notebook!)**: Please remember to check the downloaded data [Terms of use](https://www.wunderground.com/weather/api/d/terms.html?MR=1). 

In the next block, we define:

 * Path to the **folder path** containing all .json files
 * Path to the **single csv file path and name the data will be output to**. 
 
These will be the **only parameters** which are required to be modified by you. The rest of the notebook can then be executed without modifications. 

The timestamp of the tables provided by this notebook are **always** in HST in a naive format (i.e. without the timezone being explicit in the field). Modifying the output timezone to another should be a 1 line code change. To understand the rationale behind the naive HST, please see Section 5. 

Finally, Section 3.4 may be of interest to have an idea of the number of missing data points, and the end of Section 5 provides the list of all sensor types available. Do take note the data is simple formatted as a .csv, and a given sensor may have incorrect readings. It is up to you to pre-process the data..

In [3]:
from os import listdir
from os.path import isfile, join

import json
import pandas
import datetime
#import csv

# To plot days without readings
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook #function call to display the histogram in the ipython notebook
from bokeh.models import HoverTool
from collections import OrderedDict

output_notebook()

raw_folder_data_path  = 'data/KHIKAPOL19/raw'
parsed_file_data_path = 'data/KHIKAPOL19/parsed/KHIKAPOL19.csv'

Loading BokehJS ...

## 2. Data Model

The .json files obtained by the Data Acquisition notebook contain one day per file, and time resolution (i.e. min, hour) varies per station. No documentation is provided concerning resolution for a given station. 

For every .json file, the data model is as follows: 

### 2.1 There are 2 keys on the root level: 

 * Response
 * History
 
Response contains the status of the response, but does not inform, for example, if there are no readings available for the given day. A JSON will still be returned. 

### 2.2 Within `History` there are 2 other keys:

 * Observations
 * Daily Summary

### 2.3 Readings are available in the `Observations` key.

The observation key is a **list** of keys. Each **element of the list** can be of one of the 3 formats: 
 
 * reading: A single valued dictionary of the form  `<sensor_type>:<value>`
 * date: A muti-valued dictionary, contains the date from a local timezone (not sure if based on request IP or the location of the station).
 * utcdate: Similar to date, except the timestamp is guaranteed to be UTC. To avoid confusion, this timezone will be used. 
  
The following image shows an example of one observation in the **list of observations**. The bottom shows the start of a second observation (which starts on the date key again).

<img src="img/wu_json_format.png",width=400,height=400>

## 3 Parser

This section describes the ingestion of .json files, parsing into a pandas dataframe containing the wide format of the table with a header.

### 3.1 File Names 
The folder path is used to extract all file names. The following block should output 5 of these file names. If it doesn't, then the specified path is incorrect in Section 1 - Introduction's **raw_folder_data_path** variable. 

In [4]:
# Obtain the list of all file names, filter out folder names if any
file_names = [f for f in listdir(raw_folder_data_path) if isfile(join(raw_folder_data_path, f))]
print(file_names[1:5])

['20161107.json', '20161108.json', '20161109.json', '20161110.json']


### 3.2 Parser Functions

It is assumed that the types of sensors are consistent across all files **for a given station**. 

In [5]:
def parse_json_to_pandas_df(raw_folder_data_path,file_name):
    '''
        Parses a .json file into a wide-table format with readings and timestamp. 
    '''
    # Obtain full file path
    path = join(raw_folder_data_path,file_name)
    # Extract the json tree containing readings and timestamp
    df = pandas.read_json(path)['history']['observations']
    # Verify if the given day contain NO observations
    if not df:
        return None
    else: # Normalize into a table
        df = pandas.io.json.json_normalize(df)
        return(df)
    
def is_empty_day_df(df_dict):
    '''
        Takes as input a dictionary of dataframes indexed by their file name and substitute the value by Yes/No 
        if the readings exist or not for the given file. 
    '''
    is_empty_day_dict = {}
    for key, value in df_dict.items():
        if value is None:
            is_empty_day_dict[key] = 0            
        else:
            is_empty_day_dict[key] = 1
            
    df_is_empty_day =  pandas.Series(is_empty_day_dict).reset_index()
    df_is_empty_day.columns = ['date', 'exists']
    df_is_empty_day['date'] = pandas.to_datetime(df_is_empty_day['date'])
    
    return(df_is_empty_day)

### 3.3. Parse .json files into Pandas Dataframe

Each .json file is parsed into a pandas dataframe conforming to a wide table format. 

In [110]:
df_dict = {file_name.split('.')[0]:parse_json_to_pandas_df(raw_folder_data_path,file_name) for file_name in file_names}

### 3.4 Verify which days have no readings

Note the file day of the plot below is based on the name of the file from the .json. 

In [111]:
df_is_empty_day = is_empty_day_df(df_dict)

print('Number of days without any readings:'+ str(len(df_is_empty_day[(df_is_empty_day['exists']==0)]))   )

p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p.line(df_is_empty_day['date'], df_is_empty_day['exists'], color='navy', alpha=0.5)

show(p)

Number of days without any readings:95


### 3.5 Merge data frames of each day into a single data frame of readings

We first filter the days without any readings, and then merge the ones with readings.

In [112]:
for k, v in list(df_dict.items()):
    if v is None:
        del df_dict[k]
print('Number of days with readings which will be merged into the .csv file: '+str(len(df_dict)))

Number of days with readings which will be merged into the .csv file: 271


In [113]:
readings_sorted_by_date = [v for k,v in sorted(df_dict.items())]    
all_readings = pandas.concat(readings_sorted_by_date) 

## 4. Data Statistics

Full list of columns available (note this may vary according to the stations):

In [114]:
print(all_readings.columns.values)

['UV' 'date.hour' 'date.mday' 'date.min' 'date.mon' 'date.pretty'
 'date.tzname' 'date.year' 'dewpti' 'dewptm' 'heatindexi' 'heatindexm'
 'hum' 'precip_ratei' 'precip_ratem' 'precip_totali' 'precip_totalm'
 'pressurei' 'pressurem' 'softwaretype' 'solarradiation' 'tempi' 'tempm'
 'utcdate.hour' 'utcdate.mday' 'utcdate.min' 'utcdate.mon' 'utcdate.pretty'
 'utcdate.tzname' 'utcdate.year' 'wdird' 'wdire' 'wgusti' 'wgustm'
 'windchilli' 'windchillm' 'wspdi' 'wspdm']


Head of merged data frame readings:

In [115]:
all_readings.head()

,UV,date.hour,date.mday,date.min,date.mon,date.pretty,date.tzname,date.year,dewpti,dewptm,...,utcdate.tzname,utcdate.year,wdird,wdire,wgusti,wgustm,windchilli,windchillm,wspdi,wspdm
0,,00,06,00,11,"12:00 AM HST on November 06, 2016",Pacific/Honolulu,2016,65.1,18.4,...,UTC,2016,45,NE,-999.0,-1607.4,-999,-999,2.7,4.3
1,,00,06,06,11,"12:06 AM HST on November 06, 2016",Pacific/Honolulu,2016,65.1,18.4,...,UTC,2016,22,NNE,-999.0,-1607.4,-999,-999,3.7,6.0
2,,00,06,11,11,"12:11 AM HST on November 06, 2016",Pacific/Honolulu,2016,65.1,18.4,...,UTC,2016,360,North,-999.0,-1607.4,-999,-999,3.2,5.1
3,,00,06,17,11,"12:17 AM HST on November 06, 2016",Pacific/Honolulu,2016,65.1,18.4,...,UTC,2016,360,North,-999.0,-1607.4,-999,-999,3.2,5.1
4,,00,06,23,11,"12:23 AM HST on November 06, 2016",Pacific/Honolulu,2016,64.7,18.2,...,UTC,2016,22,NNE,-999.0,-1607.4,-999,-999,5.8,9.3


Tail of merged data frame readings:

In [116]:
all_readings.tail()

,UV,date.hour,date.mday,date.min,date.mon,date.pretty,date.tzname,date.year,dewpti,dewptm,...,utcdate.tzname,utcdate.year,wdird,wdire,wgusti,wgustm,windchilli,windchillm,wspdi,wspdm
223,,23,06,35,11,"11:35 PM HST on November 06, 2017",Pacific/Honolulu,2017,65.7,18.7,...,UTC,2017,292,WNW,-999.0,-1607.4,-999,-999,0.0,0.0
224,,23,06,41,11,"11:41 PM HST on November 06, 2017",Pacific/Honolulu,2017,65.5,18.6,...,UTC,2017,292,WNW,-999.0,-1607.4,-999,-999,2.2,3.5
225,,23,06,46,11,"11:46 PM HST on November 06, 2017",Pacific/Honolulu,2017,65.4,18.6,...,UTC,2017,292,WNW,-999.0,-1607.4,-999,-999,0.0,0.0
226,,23,06,52,11,"11:52 PM HST on November 06, 2017",Pacific/Honolulu,2017,65.5,18.6,...,UTC,2017,292,WNW,-999.0,-1607.4,-999,-999,0.0,0.0
227,,23,06,57,11,"11:57 PM HST on November 06, 2017",Pacific/Honolulu,2017,65.5,18.6,...,UTC,2017,292,WNW,-999.0,-1607.4,-999,-999,0.0,0.0


Total number of rows:

In [117]:
print(len(all_readings))

61314


## 5. Convert Timestamp Column from UTC to HST

There is an unecessary number of columns associated to timestamps. We merge them here into a single UTC timestamp column, which will be converted to HST in the next section. 

 * To delete: 'date.hour' 'date.mday' 'date.min' 'date.mon' 'date.pretty'
 'date.tzname' 'date.year' 'utcdate.pretty'
 * To merge into a single column timestamp: 
    * 'utcdate.year' 
    * 'utcdate.mon'
    * 'utcdate.mday'
    * 'utcdate.hour'  
    * 'utcdate.min'  
    * 'utcdate.tzname' 
    
Because time is specified in different keys under `utcdate`, the name of the attributes will contain dots. This is undesirable to access the variables using pandas anonymous functions, so we replace the dots by underline. 

In [118]:
all_readings.columns = [c.replace('.', '_') for c in all_readings.columns]
print(all_readings.columns.values)

['UV' 'date_hour' 'date_mday' 'date_min' 'date_mon' 'date_pretty'
 'date_tzname' 'date_year' 'dewpti' 'dewptm' 'heatindexi' 'heatindexm'
 'hum' 'precip_ratei' 'precip_ratem' 'precip_totali' 'precip_totalm'
 'pressurei' 'pressurem' 'softwaretype' 'solarradiation' 'tempi' 'tempm'
 'utcdate_hour' 'utcdate_mday' 'utcdate_min' 'utcdate_mon' 'utcdate_pretty'
 'utcdate_tzname' 'utcdate_year' 'wdird' 'wdire' 'wgusti' 'wgustm'
 'windchilli' 'windchillm' 'wspdi' 'wspdm']


Next, we use all the utcdate different fields from year to minute (as seconds is unavailable) and create a new timestamp column (or pandas series). 

In [119]:
timestamps = all_readings[['utcdate_year','utcdate_mon','utcdate_mday','utcdate_hour','utcdate_min']].astype(str).apply(lambda x: datetime.datetime.strptime(x.utcdate_year + x.utcdate_mon + x. utcdate_mday + x.utcdate_hour + x.utcdate_min, '%Y%m%d%H%M'), axis=1)


One frustrating limitation of native python is the naive timestamps (i.e. it does not account for timezones by default). We address this by first expliciting the series is in UTC, and then converting it to HST. 

Even more frustrating is the fact as of today Tableau can't reason timezones. As such, after the conversion we make it naive again. 

In [120]:
print(timestamps[1:5])
timestamps = timestamps.dt.tz_localize('UTC').dt.tz_convert('HST')
print(timestamps[1:5])
timestamps = timestamps.dt.tz_localize(None)
print(timestamps[1:5])

1   2016-11-06 10:06:00
2   2016-11-06 10:11:00
3   2016-11-06 10:17:00
4   2016-11-06 10:23:00
dtype: datetime64[ns]
1   2016-11-06 00:06:00-10:00
2   2016-11-06 00:11:00-10:00
3   2016-11-06 00:17:00-10:00
4   2016-11-06 00:23:00-10:00
dtype: datetime64[ns, HST]
1   2016-11-06 00:06:00
2   2016-11-06 00:11:00
3   2016-11-06 00:17:00
4   2016-11-06 00:23:00
dtype: datetime64[ns]


As we can observe above, the timestamps were correctly converted to HST (Hawaii is -10 hours in respect to UTC).

With the Series now in HST, we remove all the timestamp columns, and add our new column to the `all_readings` table. 

In [121]:
print('Before: ')
print(all_readings.columns.values)
all_readings.drop(['date_hour','date_mday','date_min','date_mon','date_pretty','date_tzname','date_year','utcdate_pretty','utcdate_year','utcdate_mon','utcdate_mday','utcdate_hour','utcdate_min','utcdate_tzname','softwaretype'], axis=1, inplace=True)
print('After: ')
all_readings['datetime'] = timestamps
print(all_readings.columns.values)
all_readings.head()

Before: 
['UV' 'date_hour' 'date_mday' 'date_min' 'date_mon' 'date_pretty'
 'date_tzname' 'date_year' 'dewpti' 'dewptm' 'heatindexi' 'heatindexm'
 'hum' 'precip_ratei' 'precip_ratem' 'precip_totali' 'precip_totalm'
 'pressurei' 'pressurem' 'softwaretype' 'solarradiation' 'tempi' 'tempm'
 'utcdate_hour' 'utcdate_mday' 'utcdate_min' 'utcdate_mon' 'utcdate_pretty'
 'utcdate_tzname' 'utcdate_year' 'wdird' 'wdire' 'wgusti' 'wgustm'
 'windchilli' 'windchillm' 'wspdi' 'wspdm']
After: 
['UV' 'dewpti' 'dewptm' 'heatindexi' 'heatindexm' 'hum' 'precip_ratei'
 'precip_ratem' 'precip_totali' 'precip_totalm' 'pressurei' 'pressurem'
 'solarradiation' 'tempi' 'tempm' 'wdird' 'wdire' 'wgusti' 'wgustm'
 'windchilli' 'windchillm' 'wspdi' 'wspdm' 'datetime']


,UV,dewpti,dewptm,heatindexi,heatindexm,hum,precip_ratei,precip_ratem,precip_totali,precip_totalm,...,tempm,wdird,wdire,wgusti,wgustm,windchilli,windchillm,wspdi,wspdm,datetime
0,,65.1,18.4,-9999,-9999,78,-99.99,-2539.7,0.00,0.0,...,22.4,45,NE,-999.0,-1607.4,-999,-999,2.7,4.3,2016-11-06 00:00:00
1,,65.1,18.4,-9999,-9999,78,-99.99,-2539.7,0.00,0.0,...,22.4,22,NNE,-999.0,-1607.4,-999,-999,3.7,6.0,2016-11-06 00:06:00
2,,65.1,18.4,-9999,-9999,78,-99.99,-2539.7,0.00,0.0,...,22.4,360,North,-999.0,-1607.4,-999,-999,3.2,5.1,2016-11-06 00:11:00
3,,65.1,18.4,-9999,-9999,78,-99.99,-2539.7,0.00,0.0,...,22.4,360,North,-999.0,-1607.4,-999,-999,3.2,5.1,2016-11-06 00:17:00
4,,64.7,18.2,-9999,-9999,77,-99.99,-2539.7,0.00,0.0,...,22.4,22,NNE,-999.0,-1607.4,-999,-999,5.8,9.3,2016-11-06 00:23:00


## 6. Save to .csv 

With the data pre-processed, we are ready to save it as a .csv. 

In [122]:
all_readings.to_csv(parsed_file_data_path)